In [1]:
from smach_utils2 import *
import matplotlib.pyplot as plt
#arm = moveit_commander.MoveGroupCommander('arm')


/home/roboworks/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
[ WARN] [1708639694.453233700]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 642.202000 according to authority /pose_integrator
[ WARN] [1708639696.165226483]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 642.859000 according to authority /pose_integrator
[ WARN] [1708639696.165266314]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 642.859000 according to authority /pose_integrator
[ WARN] [1708639696.165289968]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 642.859000 according to authority /pose_integrator
[ WARN] [1708639696.165313395]: TF_REPEATED_DATA ignoring data with redundant timestamp fo

In [ ]:
arm = moveit_commander.MoveGroupCommander('arm')

In [ ]:

arm.set_named_target('go')
arm.go()

In [ ]:
head.set_joint_values([0.0,-0.65])
rospy.sleep(0.6)
rospy.sleep(0.6)
image= cv2.cvtColor(rgbd.get_image(), cv2.COLOR_RGB2BGR)
img_msg  = bridge.cv2_to_imgmsg(image)
req      = classify_client.request_class()
req.in_.image_msgs.append(img_msg)
res      = classify_client(req)
debug_image=   cv2.cvtColor(bridge.imgmsg_to_cv2(res.debug_image.image_msgs[0]), cv2.COLOR_RGB2BGR)

In [ ]:

plt.imshow(debug_image)

In [ ]:

for i in range(len(res.poses)):
    tf_man.getTF("head_rgbd_sensor_rgb_frame")
    #tf_man.pub_static_tf(pos=[res.poses[i].position.x ,res.poses[i].position.y,res.poses[i].position.z], rot=[0,0,0,1],ref="head_rgbd_sensor_rgb_frame",point_name=res.names[i].data[4:] )   
    tf_man.pub_static_tf(pos=[res.poses[i].position.x ,res.poses[i].position.y,res.poses[i].position.z], rot=[0,0,0,1],ref="head_rgbd_sensor_rgb_frame",point_name='apple' )   
    rospy.sleep(0.3)
    tf_man.change_ref_frame_tf(res.names[i].data[4:])



In [ ]:
#omni_base.move_d_to(target_distance=0.78,target_link='apple')
pose,rot=tf_man.getTF(target_frame='hand_palm_link')
pose

In [ ]:
target_object='apple'
trans,_=tf_man.getTF(target_object,ref_frame="base_link")
trans


In [ ]:
target_object='apple'
_,rot= tf_man.getTF("base_link",ref_frame='map')

original_rot=tf.transformations.euler_from_quaternion(rot)[2]
trans,_=tf_man.getTF(target_object,ref_frame="base_link")


trans , original_rot

In [ ]:
_,rot= tf_man.getTF("base_link",ref_frame='map')
original_rot=tf.transformations.euler_from_quaternion(rot)[2]
target_object='apple'


succ = False
            
while not succ:
    
    _,rot= tf_man.getTF("base_link",ref_frame='map')
    trans,_=tf_man.getTF(target_object,ref_frame="base_link")

    trans
    eX, eY, eZ = trans
    
    eX+= -0.4
    eY+= -.06
    
    eT= tf.transformations.euler_from_quaternion(rot)[2] - original_rot #Original 
    print (eT)
    if eT > np.pi: eT=-2*np.pi+eT
    if eT < -np.pi: eT= 2*np.pi+eT
    rospy.loginfo("error: {:.2f}, {:.2f}, angle {:.2f}, target obj frame {}".format(eX, eY , eT,target_object))
    X, Y, Z = trans
    rospy.loginfo("Pose: {:.2f}, {:.2f}, angle {:.2f}, target obj frame {}".format(X, Y , eT,target_object))
    
    if abs(eX) <=0.05 :
        print ('here')
        eX = 0
    if abs(eY) <=0.05  :
        eY = 0
    if abs(eT   ) < 0.1:
        eT = 0
    succ =  eX == 0 and eY == 0 and eT==0

    
    omni_base.tiny_move( velX=0.2*+eX,velY=0.3*eY, velT=-eT,std_time=0.2, MAX_VEL=0.3) 


In [ ]:
res.names

In [ ]:
head.set_joint_values([0.0,0.0])
clear_octo_client()

hand_cam= HAND_RGB()


In [ ]:
#clear_octo_client()

pickup_pose=[min(trans[2],0.66),-1.2,0.0,-1.9, 0.0, 0.0]
#pickup_pose=[0.65,-1.2,0.0,-1.9, 0.0, 0.0]
succ= arm.go(pickup_pose)
gripper.open()


In [ ]:

image= cv2.cvtColor(hand_cam.get_image(), cv2.COLOR_RGB2BGR)
img_msg  = bridge.cv2_to_imgmsg(image)
req      = classify_client.request_class()
req.in_.image_msgs.append(img_msg)
res      = classify_client(req)
debug_image=   cv2.cvtColor(bridge.imgmsg_to_cv2(res.debug_image.image_msgs[0]), cv2.COLOR_RGB2BGR)

plt.imshow(debug_image)

## [(323, 266), (280, 215)]  at pkcp pose


In [ ]:
abs(eX) > 0.05 or abs(eY) > 0.05

In [ ]:
eX,eY= 0.3,0.3 

while abs(eX) > 0.05 or abs(eY) > 0.05:
    image= cv2.cvtColor(hand_cam.get_image(), cv2.COLOR_RGB2BGR)
    img_msg  = bridge.cv2_to_imgmsg(image)
    req      = classify_client.request_class()
    req.in_.image_msgs.append(img_msg)
    res      = classify_client(req)


    for i in range(len(res.names)):
        if res.names[i].data== '013_apple' or res.names[i].data== '017_orange':
            bbM=res.pt_min.data[2*i:2*i+2]
            bbm=res.pt_max.data[2*i:2*i+2]
            bbx=[bbM,bbm]
    print(bbx)
    eY=(bbx[1][1]-215)/215    #experience value
    #print(f'error relative to gripper cam {eY}')


    #eX=(bbx[0][0]-323)/323
    eX=(bbx[0][0]-291)/291
    eY=(bbx[1][1]-226)/226
    #eY=(bbx[1][1]-215)/215    #experience value

    print(f'error relative to gripper cam ex={eX} ey={eY}')
    omni_base.tiny_move(velX=eX, velY=-eY, MAX_VEL=0.05) 
print(f'error relative to gripper cam ex={eX} ey={eY}:::::: Error within tolerance')

In [ ]:
clear_octo_client()
pose,_=tf_man.getTF(target_frame='hand_palm_link',ref_frame='apple')
av=arm.get_current_joint_values()
print (av,'av')
av=arm.get_current_joint_values()
av[0]+=0.07-pose[2]
#av[0]+= -0.17###LAB
#av[0]+= -0.35###GAZ
succ=arm.go(av)
rospy.sleep(0.5)
gripper.close(force=0.06)

In [ ]:
av=arm.get_current_joint_values()
print (av,'av')
av[0]+= 0.15
succ=arm.go(av)
omni_base.tiny_move(velX=-0.3, std_time=4.0)
arm.set_named_target('go')
arm.go()
#tf_man.pub_static_tf(point_name='apple')

In [ ]:
pose,rot=tf_man.getTF(target_frame='hand_palm_link',ref_frame='apple')
pose[2]

In [ ]:
#clear_octo_client()

pickup_pose=[0.65,-1.2,0.0,-1.9, 0.0, 0.0]
succ= arm.go(pickup_pose)
gripper.open()


[ WARN] [1708639717.084610808]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 651.480000 according to authority /pose_integrator
[ WARN] [1708639717.084657344]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 651.480000 according to authority /pose_integrator
[ WARN] [1708639717.084692053]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 651.480000 according to authority /pose_integrator
[ WARN] [1708639717.084740851]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 651.480000 according to authority /pose_integrator
[ WARN] [1708639717.084789029]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 651.480000 according to authority /pose_integrator
[ WARN] [1708639717.084812912]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 651.480000 according to authority /pose_integrator
[ WARN] [1708639717.34504468

In [8]:
goal

timeout: 0.0
target: 
  data: "apple"

[ WARN] [1708639724.363636744]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 654.267000 according to authority /pose_integrator
[ WARN] [1708639724.363699148]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 654.267000 according to authority /pose_integrator
[ WARN] [1708639724.363755199]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 654.267000 according to authority /pose_integrator
[ WARN] [1708639724.363795801]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 654.267000 according to authority /pose_integrator
[ WARN] [1708639724.363831043]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 654.267000 according to authority /pose_integrator
[ WARN] [1708639724.363862831]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 654.267000 according to authority /pose_integrator
[ WARN] [1708639724.45798258